In [11]:
import os
import numpy as np 
import pandas as pd 
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows",None)
from sklearn import preprocessing
import matplotlib 
matplotlib.style.use('ggplot')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [59]:
PROJECT = !gcloud config list --format 'value(core.project)'
PROJECT = PROJECT[0]
BUCKET = PROJECT
REGION = "us-central1"

os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

In [12]:
hf_df = pd.read_csv(r'/home/jupyter/heart-failure/heart.csv')

In [13]:
hf_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [14]:
hf_df.HeartDisease.value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

In [15]:
Train, Test = train_test_split(hf_df, test_size=.2, stratify=hf_df.HeartDisease)

In [16]:
Train.HeartDisease.value_counts(), Test.HeartDisease.value_counts()

(1    406
 0    328
 Name: HeartDisease, dtype: int64,
 1    102
 0     82
 Name: HeartDisease, dtype: int64)

In [64]:
def oversample_df(df,stratvar, addsamples,test_size,random_state):
    #oversample the dataframe to get to ~1000 samples for automl.. keep dupes in same split
    
    Train, Test = train_test_split(hf_df, test_size=test_size, stratify=hf_df[stratvar],random_state=random_state)
    
    #define the amount to upsample in each group
    train_upsample_num = addsamples * (1-test_size)
    test_upsample_num = addsamples * test_size
    
    #split train and test to positive and negative classses and sample proportionately
    perc = Train[stratvar].value_counts()[1]/Train[stratvar].value_counts().sum()
    
    train_pos_samples = int(np.around(Train[stratvar].value_counts()[1]/Train[stratvar].value_counts().sum() * train_upsample_num,0))
    train_neg_samples = int(np.around(train_upsample_num - train_pos_samples))
    
    test_pos_samples = int(np.around(Test[stratvar].value_counts()[1]/Test[stratvar].value_counts().sum() * test_upsample_num,0))
    test_neg_samples = int(np.around(test_upsample_num - test_pos_samples))
    
    
    Train_pos_upsample = Train[Train[stratvar]==1].sample(train_pos_samples,random_state=random_state)
    Train_neg_upsample = Train[Train[stratvar]==0].sample(train_neg_samples,random_state=random_state)
    
    Train_synth = pd.concat([Train,Train_pos_upsample,Train_neg_upsample])
    Train_synth['tt_split'] = 'TRAIN'
    
    Test_pos_upsample = Test[Test[stratvar]==1].sample(test_pos_samples)
    Test_neg_upsample = Test[Test[stratvar]==0].sample(test_neg_samples)
    
    Test_synth = pd.concat([Test,Test_pos_upsample,Test_neg_upsample])
    
    #split again to validation
    Test_synth_sub, Validation_synth = train_test_split(Test_synth, test_size=.5, stratify=Test_synth[stratvar],random_state=random_state)
    
    Test_synth_sub['tt_split'] = 'TEST'
    Validation_synth['tt_split'] = 'VALIDATE'

    train_test_out = pd.concat([Train_synth,Test_synth_sub,Validation_synth])
    
    return train_test_out
    
    

In [65]:
hd_df_oversample = oversample_df(hf_df,stratvar='HeartDisease',
                                 addsamples=1000-len(hf_df),test_size=.2,random_state=42)

In [66]:
hd_df_oversample.HeartDisease.value_counts().sum()

1000

In [67]:
hd_df_oversample.tt_split.value_counts()

TRAIN       800
TEST        100
VALIDATE    100
Name: tt_split, dtype: int64

In [68]:
heart_disease_bucket = F'gs://{BUCKET}/heart_failure/heart_oversample.csv'

In [70]:
hd_df_oversample.to_csv(heart_disease_bucket,index=False)